In [99]:
import zipfile
import gensim
import numpy as np
import json
import random
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.cluster import AgglomerativeClustering, SpectralClustering, DBSCAN
#import classla
#import fasttext.util

In [101]:
#Some utility functions which are used
def cos_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def remove_diplicate_sentences(a):
    b_set = set(map(tuple,a))  #need to convert the inner lists to tuples so they are hashable
    b = list(map(list,b_set)) #Now convert tuples back into lists (maybe unnecessary?)
    return b

## Loading word2vec skipgram model

In [5]:
repository = "../../"
with zipfile.ZipFile(repository + "/67.zip", "r") as archive:
    stream = archive.open("model.txt")
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=False, unicode_errors='replace')

In [6]:
#testing
result = model.similar_by_word("legenda")
most_similar_key, similarity = result[0]  
print(f"{most_similar_key}: {similarity:.4f}")

picokih: 0.6543


## Loading all sentences

In [9]:
with open('../ccGigaFida/results/data.json') as json_file:
    data = json.load(json_file)
with open('../ccGigaFida/results/data_lema.json') as json_file:
    data_lema = json.load(json_file)
with open('../ccGigaFida/results/data_pos.json') as json_file:
    data_len = json.load(json_file)

words = np.load("../ccGigaFida/words.npy")
words

array(['leto', 'dan', 'konec', 'svet', 'stran', 'mesto', 'šola', 'ura',
       'beseda', 'pot', 'red', 'zakon', 'zadeva', 'srce', 'tema',
       'resnica', 'moški', 'vloga', 'kraj', 'stanje', 'škoda', 'film',
       'večer', 'vrh', 'jutro', 'kazen', 'oblast', 'račun', 'novica',
       'milijon', 'par', 'krog', 'tip', 'punca', 'sila', 'vir', 'las',
       'akcija', 'meter', 'prst', 'kri', 'stik', 'grad', 'znak', 'lik',
       'direktor', 'vodja', 'raven', 'kolo', 'rob', 'gost', 'duh',
       'praznik', 'vest', 'korist', 'vedenje', 'tek', 'kup', 'otok',
       'razstava', 'bitje', 'motor', 'karta', 'nevarnost', 'hitrost',
       'kos', 'zob', 'stroj', 'kamen', 'župan', 'šef', 'vrtec', 'kot',
       'deček', 'avgust', 'tok', 'jezero', 'klop', 'čelo', 'hip', 'kupec',
       'pojav', 'čaj', 'postava', 'dolg', 'standard', 'jesen', 'rak',
       'grob', 'plus', 'les', 'vez', 'polica', 'minus', 'plan', 'posoda',
       'restavracija', 'jok', 'krilo', 'sol', 'rod', 'stres', 'trditev',
       'f

In [117]:
for keyword in ['drag']:
    all_sentences=data[keyword][0:1000]
    all_sentences_lema=data_lema[keyword][0:1000]
    
    #Remove duplicate sentences if any
    #all_sentences=remove_diplicate_sentences(all_sentences)
    #all_sentences_lema=remove_diplicate_sentences(all_sentences_lema)
    
    all_embeddings=np.zeros((len(all_sentences),100))
    
    for i in range(len(all_sentences)): #iterate through the sentences for the given keyword
        keyword_position = all_sentences_lema[i].index(keyword)
        #all_sentences[i].pop(keyword_position)
        
        centroid=np.zeros(100)
        words_added=0
        
        for word_index, word in enumerate(all_sentences[i]): #iterate through the words in the sentence
            try:
                if word_index!=keyword_position:
                    centroid+=model[word.lower()] 
                    words_added+=1
            except:
                ...
                
        if words_added==0: #there are wierd sentences where only the keyword is the whole sentence
            continue
            
        centroid/=words_added
        all_embeddings[i,:]=centroid
        
    
    break #this break means that we terminate on the first word
    
#~np.all(all_embeddings == 0, axis=1) this checks if there are any only-zero rows meaning that we did not have any word embedding for that sentence

KeyError: 'drag'

In [113]:
distances={}
for i in range(all_embeddings.shape[0]):
    for j in range(i+1,all_embeddings.shape[0],1):
        distances[str(i)+'-'+str(j)] = cos_similarity(all_embeddings[i,:], all_embeddings[j,:])

<ipython-input-101-cca5a08a4d16>:3: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


In [114]:
distances = dict(sorted(distances.items(), key=lambda item: item[1]))

## Analyze worst 50 distances

In [115]:
distances_keys = list(distances.keys())
already_were = [] #to je zato ker se stavke ponavljajo in ne zanimajo nas iste kombinacije znova
count=0
for key in distances_keys[0:100]:
    first, second = map(int, key.split('-'))
    first_sent, second_sent = ' '.join(all_sentences[first]), ' '.join(all_sentences[second])
    
    if first_sent+second_sent in already_were:
        continue
    
    count+=1
    already_were.append(first_sent+second_sent)
    print(first_sent+'\n'+second_sent)
    print('-----')
    if count==10:
        break

Med najbolj znanimi misijonarji slovenskega rodu je bil vsekakor Ignacij Knoblehar misijonar v Sudanu
TOČENJE MEDU
-----
Posamezni deli so namreč danes zelo nepovezani med sabo
TOČENJE MEDU
-----
Med gradbenimi podvigi naj omenimo čudovit baročni dvorec v Goričanah ki sta ga zidala in opremila škofa Oton Friderik grof Buchheim in Jožef grof Rabatta novo ljubljansko stolnico ki jo je leta 1707 posvetil škof Ferdinand Franc grof Kuenburg in veličastno cerkev sv. Mohorja in Fortunata v Gornjem Gradu ki jo je sredi 18. stoletja dal postaviti škof Ernest Amadej grof Attems
TOČENJE MEDU
-----
Lačnova že med 50
MED Sporazum
-----
Med najpomembnejše pionirje atomskih bomb spadata zlasti ameriški fizik J. Robert Oppenheimer znanstveni vodja projekta Manhattan
TOČENJE MEDU
-----
Ko so sodili nacističnim zločincem v Nürnbergu so bili med sodniki tudi sovjetski ki bi morali najprej pomesti pred svojim pragom
TOČENJE MEDU
-----
Sodelovanje med materjo in sinom je bilo oteženo delno zaradi generacij

## Analyze best 50 distances

In [116]:
distances_keys = list(distances.keys())
already_were = [] #to je zato ker se stavke ponavljajo in ne zanimajo nas iste kombinacije znova
count=0
for key in distances_keys[-1000:]:
    first, second = map(int, key.split('-'))
    first_sent, second_sent = ' '.join(all_sentences[first]), ' '.join(all_sentences[second])
    if first_sent+second_sent in already_were:
        continue
    
    count+=1
    already_were.append(first_sent+second_sent)
    print(first_sent+'\n'+second_sent)
    print('-----')
    if count==10:
        break

Vroča diskusija se je končala s sklepom da bo Odbor pozdravil ustanovitev SDZ ne bo pa med soustanovitelji
Tiskovni predstavnik Evropske komisije mu je takrat odgovoril da je kandidatk za članstvo že veliko a med njimi ni Izraela in Rusije
-----
Ko je bil v generalni hiši v Rimu med leti 1944 in 1946 je bil tam iz Slovenije neki komunist za katerega pater Tomaž ni vedel kakšnih nazorov je
Mešano množico v kateri je bilo tudi veliko takih ki so čisto naključno med sprehodom po Izoli ugotovili da se na Manziolijevem trgu nekaj dogaja je navdušil tudi Tartinijev Koncert za trobento v katerem je kot solist nastopil Stanko Arnold
-----
V tem času pa je pri izoblikovanju meril sodelovalo veliko strokovnjakov med njimi prav precej takih ki so vse svoje življenje posvetili psom
Na ta način ne bo časa da bi kakšnega med slovenskimi škofi pohvalil ali posebno pokaral
-----
Med Črnovrščani je bilo pred časom slišati da naj bi ga kupilo šolsko ministrstvo za seminarje šolnikov in za obšolske dejav

## Construct similarity distance matrix

In [73]:
#print(all_embeddings.shape)
distance_matrix=(pairwise_distances(all_embeddings,metric="cosine"))
distance_matrix

array([[0.        , 0.13731799, 0.22660908, ..., 0.32688616, 0.24738726,
        0.27633132],
       [0.13731799, 0.        , 0.12635066, ..., 0.28019894, 0.18324466,
        0.17706561],
       [0.22660908, 0.12635066, 0.        , ..., 0.23151193, 0.20322849,
        0.1742427 ],
       ...,
       [0.32688616, 0.28019894, 0.23151193, ..., 0.        , 0.32300266,
        0.32060998],
       [0.24738726, 0.18324466, 0.20322849, ..., 0.32300266, 0.        ,
        0.29758845],
       [0.27633132, 0.17706561, 0.1742427 , ..., 0.32060998, 0.29758845,
        0.        ]])

## Clustering

In [81]:
clusters = AgglomerativeClustering(affinity='precomputed', linkage='complete', n_clusters=2).fit(distance_matrix)
clusters.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [82]:
SpectralClustering(2).fit_predict(distance_matrix)

C:\Users\marko\anaconda3\envs\ids-clone\lib\site-packages\sklearn\cluster\_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [95]:
dbscan = DBSCAN(metric='cosine', eps=0.4, min_samples=3).fit(all_embeddings)  # you can change these parameters, given just for example 
labels = dbscan.labels_ # where X - is your matrix, where each row corresponds to one document (line) from the docs, you need to cluster 
#cluster_labels
no_clusters = len(np.unique(labels) )
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

labels
#all_sentences[np.where(labels == 1)]
for i,label in enumerate(labels):
    if label==1:
        print(all_sentences[i])

Estimated no. of clusters: 2
Estimated no. of noise points: 0
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
['Priznanje', 'Podjetnik', 'leta']
